<center> <img src="https://miro.medium.com/v2/resize:fit:1200/1*lbDXL0IuitCRz4mpZ7MmfQ.png" width=55% > </center>

<br><br>

<center> 
    <font size="6">Final Lab (Part 2): Image Classification using Convolutional Neural Networks </font>
</center>
<center> 
    <font size="4">Computer Vision 1 University of Amsterdam</font> 
</center>
<center> 
    <font size="4">Due 23:59PM, 24th October, 2025 (Amsterdam time)</font> 
</center>

<br><br>

***

<br><br>

<center>

Student1 ID:  \
Student1 Name: 

Student2 ID: \
Student2 Name: 

Student3 ID: \
Student3 Name: 

( Student4 ID: \
Student4 Name: )

</center>

### **Coding Guidelines**

Your code must be handed in this Jupyter notebook, renamed to **StudentID1_StudentID2_StudentID3.ipynb** before the deadline by submitting it to the Canvas Final Lab: Image Classification Assignment. Please also fill out your names and IDs above.

For full credit, make sure your notebook follows these guidelines:

- Please express your thoughts **concisely**. The number of words does not necessarily correlate with how well you understand the concepts.
- Understand the problem as much as you can. When answering a question, provide evidence (qualitative and/or quantitative results, references to papers, figures, etc.) to support your arguments. Not everything might be explicitly asked for, so think about what might strengthen your arguments to make the notebook self-contained and complete.
- Tables and figures must be accompanied by a **brief** description. Add a number, a title, and, if applicable, the name and unit of variables in a table, and name and unit of axes and legends in a figure.

**Late submissions are not allowed.** Assignments submitted after the strict deadline will not be graded. In case of submission conflicts, TAs’ system clock is taken as reference. We strongly recommend submitting well in advance to avoid last-minute system failure issues.

**Environment:** Since this is a project-based assignment, you are free to use any feature descriptor and machine learning tools (e.g., K-means, SVM). You should use Python for your implementation. You are free to use any Python library for this assignment, but make sure to provide a conda environment file!

**Plagiarism Note:** Keep in mind that plagiarism (submitted materials which are not your work) is a serious offense and any misconduct will be addressed according to university regulations. This includes using generative tools such as ChatGPT.

**Ensure that you save all results/answers to the questions (even if you reuse some code).**

### **Report Preparation**

Your tasks include the following:

1. **Report Preparation:** For both parts of the final project, students are expected to prepare a report. The report should include all details on implementation approaches, analysis of results for different settings, and visualizations illustrating experiments and performance of your implementation. Grading will be based on the report, so it should be as self-contained as possible. If the report contains faulty results or ambiguities, TAs can refer to your code for clarification. 

2. **Explanation of Results:** Do not just provide numbers without explanation. Discuss different settings to show your understanding of the material and processes involved.

3. **Quantitative Evaluation:** For quantitative evaluation, you are expected to provide the results based on performance (accuracy, learning loss and learning curves). 

4. **Aim:** Understand the basic Image Classification pipeline using Convolutional Neural Nets (CNN's).

5. **Working on Assignments:** Students should work in assigned groups for **two** weeks. Any questions can be discussed on ED.

    - **Submission:** Submit your source code and report together in a zip file (`ID1_ID2_ID3_part2.zip`). The report should be a maximum of 10 pages (single-column, including tables and figures, excluding references and appendix). Express thoughts concisely. Tables and figures must be accompanied by a description. Number them and, if applicable, name variables in tables, and label axes in figures.

6. **Hyperparameter Search:** In your experiments, remember to perform a hyperparameter search to find the optimal settings for your model(s). Clearly document the search process, the parameters you explored, and how they influenced the performance of your model.

8. **Format and Testing:** The report should be in **PDF format**, and the code in **.ipynb format**. Test that all functionality works as expected in the notebook.

### **Overview**

- [Section 1: Image Classification on CIFAR-100 (0 points)](#section-1)
- [Section 2: Visualizing CIFAR-100 Classes and Subclasses (3 points)](#section-2)
- [Section 3: TwoLayerNet Architecture (2 points)](#section-3)
- [Section 4: ConvNet Architecture (2 points)](#section-4)
- [Section 5: Preparation of Training (7 points)](#section-5)
- [Section 6: Training the Networks (5 points)](#section-6)
- [Section 7: Setting Up the Hyperparameters (14 points)](#section-7)
- [Section 8: Visualizing the STL-10 Dataset and Preparing the Data Loader (3 points)](#section-8)
- [Section 9: Fine-tuning ConvNet on STL-10 (14 points)](#section-9)
- [Section 10: Bonus Challenge (optional)](#section-10)
- [Section X: Individual Contribution Report (Mandatory)](#section-x)

### **Section 1: Image Classification on CIFAR-100 (0 points)**

The goal of this lab is to implement an image classification system using Convolutional Neural Networks (CNNs) that can identify objects from a set of classes in the [CIFAR-100 dataset](https://www.cs.toronto.edu/~kriz/cifar.html). You will implement and compare two different architectures: a simple two-layer network and a ConvNet based on the LeNet architecture.

The CIFAR-100 dataset contains 32x32 pixel RGB images, categorized into 100 different classes. The dataset will be automatically downloaded and loaded using the code provided in this notebook.

You will train and test your classification system using the entire CIFAR-100 dataset. Ensure that the test images are excluded from training to maintain a fair evaluation of the model's performance.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from PIL import Image
from torchinfo import summary
from torch.utils.data import DataLoader, Dataset, random_split

# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  
])

# Load the CIFAR-100 training set
train_set = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)

# Load the CIFAR-100 test set
test_set = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

# Create data loaders for the entire CIFAR-100 dataset
train_data_loader = DataLoader(train_set, shuffle=True)
test_data_loader = DataLoader(test_set, shuffle=False)

# Define CIFAR-100 superclasses and their subclasses
superclasses = {
    'aquatic mammals': ['beaver', 'dolphin', 'otter', 'seal', 'whale'],
    'fish': ['aquarium_fish', 'flatfish', 'ray', 'shark', 'trout'],
    'flowers': ['orchid', 'poppy', 'rose', 'sunflower', 'tulip'],
    'food containers': ['bottle', 'bowl', 'can', 'cup', 'plate'],
    'fruit and vegetables': ['apple', 'mushroom', 'orange', 'pear', 'sweet_pepper'],
    'household electrical devices': ['clock', 'keyboard', 'lamp', 'telephone', 'television'],
    'household furniture': ['bed', 'chair', 'couch', 'table', 'wardrobe'],
    'insects': ['bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach'],
    'large carnivores': ['bear', 'leopard', 'lion', 'tiger', 'wolf'],
    'large man-made outdoor things': ['bridge', 'castle', 'house', 'road', 'skyscraper'],
    'large natural outdoor scenes': ['cloud', 'forest', 'mountain', 'plain', 'sea'],
    'large omnivores and herbivores': ['camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo'],
    'medium-sized mammals': ['fox', 'porcupine', 'possum', 'raccoon', 'skunk'],
    'non-insect invertebrates': ['crab', 'lobster', 'snail', 'spider', 'worm'],
    'people': ['baby', 'boy', 'girl', 'man', 'woman'],
    'reptiles': ['crocodile', 'dinosaur', 'lizard', 'snake', 'turtle'],
    'small mammals': ['hamster', 'mouse', 'rabbit', 'shrew', 'squirrel'],
    'trees': ['maple_tree', 'oak_tree', 'palm_tree', 'pine_tree', 'willow_tree'],
    'vehicles 1': ['bicycle', 'bus', 'motorcycle', 'pickup_truck', 'train'],
    'vehicles 2': ['lawn_mower', 'rocket', 'streetcar', 'tank', 'tractor']
}

# List of all CIFAR-100 classes
classes = ('apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 
           'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle',
           'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur',
           'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard',
           'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain',
           'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree',
           'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea',
           'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 
           'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 
           'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm')

# Create a mapping of class names to their indices
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}

# Create a mapping of superclasses to their corresponding class indices
superclass_to_indices = {supcls: [class_to_idx[cls] for cls in subclasses] for supcls, subclasses in superclasses.items()}

print("Data loaders for CIFAR-100 are ready for use.")

<a id="section-2"></a>
### **Section 2: Visualizing CIFAR-100 Classes and Subclasses (3 points)**

In this section, you will implement a function to visualize the CIFAR-100 dataset, including **all** superclasses and their corresponding subclasses. Your implementation should provide a clear and organized overview of the dataset's diversity.

You add the figure(s) to appendix of your report and refer to it in the main text.

In [ ]:
# YOUR CODE HERE

# Fetch the data without the normalization
visual_Set = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transforms.ToTensor())

found = [] # List of all subclasses for which we have an image
images = {} # Lookup for the subclass images
idx = 0 # Current index in the visual dataset

# Loop through the data until we have an image for all subclasses
while(len(found) < 100):
    img, target = visual_Set.__getitem__(idx)

    # If this is a new subclass, store it
    if not (target in found):
        found.append(target)
        images[target] = img.permute(1, 2, 0)
    
    idx += 1

# Plot the images
fig, axs = plt.subplots(nrows=len(superclasses), ncols=int(len(classes)/len(superclasses)), figsize=(5,20), constrained_layout=True)
for row, superclass in enumerate(superclasses):

    # Plot all the subclasses
    for column, subclass in enumerate(superclasses[superclass]):
        axs[row, column].axis('off')
        axs[row, column].imshow(images[class_to_idx[subclass]])
        axs[row, column].set_title(subclass)

    # Add superclass name
    heigth = 1.0 - (row + 0.5) / len(superclasses)
    fig.text(-0.5, heigth, superclass, ha='left', va='center', weight='bold')

plt.show()

<a id="section-3"></a>
### **Section 3: TwoLayerNet Architecture (2 points)**

In this section, you will implement the architecture of a fully connected neural network called `TwoLayerNet`, consisting of two fully connected layers with a ReLU activation in between. The network accepts an input size of 3x32x32 (CIFAR-100 image), a specified hidden layer size, and the number of output classes. In the `__init__` method, define the first fully connected layer that maps the input size to the hidden size, and the second fully connected layer that maps the hidden size to the number of classes. 

Ensure to call the parent class constructor using `super(TwoLayerNet, self).__init__()`. In the `forward` method, flatten the input tensor, pass it through the first layer with ReLU activation, and then through the second layer to obtain the final scores.

**Note:** You are allowed to modify the provided function definitions as needed.

In [ ]:
class TwoLayerNet(nn.Module):

    def __init__(self, input_size, hidden_size, num_classes):
        '''
        Initializes the two-layer neural network model.

        Args:
            input_size (int): The size of the input features.
            hidden_size (int): The size of the hidden layer.
            num_classes (int): The number of classes in the dataset.
        '''

        # YOUR CODE HERE
        super(TwoLayerNet, self).__init__()

        self.flat = nn.Flatten()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x: torch.Tensor):
        '''
        Defines the forward pass of the neural network.

        Args:
            x (torch.Tensor): The input tensor.

        Returns:
            torch.Tensor: The output tensor.
        '''

        # YOUR CODE HERE

        # Apply the two layers with relu inbetween
        x = self.flat(x)
        x = self.layer1(x)
        x = nn.functional.relu(x)
        x = self.layer2(x)

        return x



<a id="section-4"></a>
### **Section 4: ConvNet Architecture (2 points)**

In this section, you will implement a convolutional neural network inspired by the structure of [LeNet-5](https://ieeexplore.ieee.org/document/726791). The network processes color images using three convolutional layers followed by two fully connected layers. Since you need to feed color images into this network, determine the kernel size of the first convolutional layer. Additionally, calculate the number of trainable parameters in the "F6" layer, providing the calculation process.

In [ ]:
class ConvNet(nn.Module):

    def __init__(self, num_classes):
        '''	
        Initializes the convolutional neural network model.

        Args:
            num_classes (int): The number of classes in the dataset.
        '''

        # YOUR CODE HERE
        super(ConvNet, self).__init__()

        self.layer_C1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(5, 5))
        self.layer_S2 = nn.AvgPool2d(kernel_size=(2, 2))
        self.layer_C3 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5))
        self.layer_S4 = nn.AvgPool2d(kernel_size=(2, 2))
        self.layer_C5 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5, 5))
        self.flat = nn.Flatten()
        self.layer_F6 = nn.Linear(120, 84)
        self.gaussian_connection = nn.Linear(84, num_classes)

    def forward(self, x):
        '''
        Defines the forward pass of the neural network.

        Args:
            x (torch.Tensor): The input tensor.

        Returns:
            torch.Tensor: The output tensor.
        '''

        # YOUR CODE HERE

        # Apply all the layers and the activation functions ("sigmoid squashing function" in the paper)
        x = self.layer_C1(x)
        x = nn.functional.tanh(x) * 1.7159

        x = self.layer_S2(x)

        x = self.layer_C3(x)
        x = nn.functional.tanh(x) * 1.7159

        x = self.layer_S4(x)

        x = self.layer_C5(x)
        x = nn.functional.tanh(x) * 1.7159

        x = self.flat(x)

        x = self.layer_F6(x)
        x = nn.functional.tanh(x)

        x = self.gaussian_connection(x)

        return x


<a id="section-5"></a>
### **Section 5: Preparation of Training (7 points)**

In this section, you will create a custom dataset class to load the CIFAR-100 data, define a transform function for data augmentation, and set up an optimizer for training. While the previous section utilized the built-in CIFAR-100 class from `torchvision`, in practice, you often need to prepare datasets manually. Here, you will implement the `CIFAR100_loader` class to handle the dataset and use `DataLoader` to make it iterable. You will also define a transform function for data augmentation and an optimizer for updating the model's parameters.

In [ ]:
class CIFAR100_loader(Dataset):
    
    def __init__(self, root, train=True, transform=None, download=False):
        '''
        Initializes the CIFAR-100 dataset loader.

        Args:
            root (str): The root directory to store the dataset.
            train (bool): If True, loads the training data; otherwise, loads the test data.
            transform (callable, optional): The data transformations to apply.
            download (bool): If True, downloads the dataset if it is not already available.
        '''
    
        # YOUR CODE HERE
        self.data_set = torchvision.datasets.CIFAR100(root=root, train=train, download=download, transform=transform)

    def __len__(self):
        '''
        Returns the number of samples in the dataset.

        Returns:
            int: The number of samples in the dataset.
        '''

        # YOUR CODE HERE
        return len(self.data_set)


    def __getitem__(self, idx):
        '''
        Retrieves a sample from the dataset at the specified index.

        Args:
            idx (int): The index of the sample to retrieve.

        Returns:
            tuple: A tuple containing the image and label tensors.
        '''

        # YOUR CODE HERE
        return self.data_set[idx]

In [ ]:
def create_transforms():
    '''
    Creates the data transformations for the CIFAR-100 dataset.

    Returns:
        torchvision.transforms.Compose: The data transformations for the dataset.
    '''

    # YOUR CODE HERE

    # Values for Normalize come from https://gist.github.com/weiaicunzai/e623931921efefd4c331622c344d8151
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))  
    ])

    return transform

In [ ]:
def create_optimizer(model, learning_rate=0.001):
    '''
    Creates an optimizer for the model.

    Args:
        model (torch.nn.Module): The neural network model.
        learning_rate (float): The learning rate for the optimizer.

    Returns:
        torch.optim.Adam: The optimizer for the model.
    '''

    # YOUR CODE HERE

    return torch.optim.Adam(model.parameters(), lr=learning_rate)

<a id="section-6"></a>
### **Section 6: Training the Networks (5 points)**

In this section, you will complete the `train` function and use it to train both the `TwoLayerNet` and `ConvNet` models. You will use the custom `CIFAR100_loader`, transform function, and optimizer function that you implemented. The goal is to compare the performance of the two models on the CIFAR-100 dataset.

In [ ]:
def validate(net, testloader):
    '''
    Validates the model on the test dataset.

    Args:
        net (torch.nn.Module): The neural network model.
        testloader (torch.utils.data.DataLoader): The data loader for the test dataset.

    Returns:
        float: The accuracy of the model on the test dataset.
    '''

    # Set the model to evaluation mode
    net.eval()

    # Determine the device to run the model on
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    correct, total = 0, 0

    # Disable gradient computation
    with torch.no_grad():

        # Iterate over the test dataset
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the test images: {accuracy:.2f} %')

    return accuracy

In [ ]:
def validate_per_class(net, testloader, classes):
    '''
    Validates the model on the test dataset per class.

    Args:
        net (torch.nn.Module): The neural network model.
        testloader (torch.utils.data.DataLoader): The data loader for the test dataset.
        classes (tuple): The tuple of class names.

    Returns:
        None
    '''

    # Set the model to evaluation mode
    net.eval()

    # Determine the device to run the model on
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    class_correct = [0. for _ in range(len(classes))]
    class_total = [0. for _ in range(len(classes))]

    # Disable gradient computation
    with torch.no_grad():

        # Iterate over the test dataset
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_predictions = (predicted == labels).squeeze()

            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += correct_predictions[i].item()
                class_total[label] += 1

    for i, class_name in enumerate(classes):
        accuracy = 100 * class_correct[i] / class_total[i] if class_total[i] > 0 else 0
        print(f'Accuracy of {class_name:5s} : {accuracy:.2f} %')

In [ ]:
def train(net, train_loader, criterion, optimizer, epochs=100):
    '''
    Trains the neural network model.

    Args:
        net (torch.nn.Module): The neural network model.
        train_loader (torch.utils.data.DataLoader): The data loader for the training dataset.
        criterion (torch.nn.Module): The loss function.
        optimizer (torch.optim.Optimizer): The optimizer for the model.
        epochs (int): The number of epochs to train the model.

    Returns:
        None
    '''

    # YOUR CODE HERE

    # Determine the device to run the model on
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Print the training device
    print("Training on device", device)

    # Set the model to train mode
    net.to(device)
    net.train()

    # Store the average losses of each epoch
    losses = []
    accuracies = []
    

    # Train the network
    for epoch in range(epochs):

        # Store the total loss and correct predictions for the epoch to track training progress
        total_loss = 0
        correct, total = 0, 0

        for inputs, labels in train_loader:

            # Move data to correct device
            inputs, labels = inputs.to(device), labels.to(device)

            # Determine logits
            outputs = net(inputs)
            
            # Calculate the loss
            loss = criterion(outputs, labels)

            # Back propegate
            loss.backward()

            # Update weigths
            optimizer.step()

            # Remove the gradients for the next iteration
            net.zero_grad()
            optimizer.zero_grad()

            # Update the total loss and the predictions
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Output the average loss and the accuracy of the epoch
        avg_loss = total_loss / len(train_loader)
        accuracy = 100 * correct / total
        losses.append(avg_loss)
        accuracies.append(accuracy)
        print(f"Epoch {epoch} finished with {round(avg_loss, 3)} loss and {round(accuracy, 1)} accuracy.")

    # Plot the learning curve
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20,8), constrained_layout=True)
    axs[0].plot(losses, label='Training loss')
    axs[0].set_title('Average loss per epoch versus epoch.')
    axs[0].set_xlabel('Epoch')
    axs[0].set_ylabel('Average loss')
    axs[0].legend()

    axs[1].plot(accuracies, label='Training accuracy')
    axs[1].set_title('Training accuracy per epoch versus epoch.')
    axs[1].set_xlabel('Epoch')
    axs[1].set_ylabel('Accuracy')
    axs[1].legend()
    fig.show()


First, initialize the datasets and data loaders for both models.

In [ ]:
# YOUR CODE HERE

# Set the seed for reproducability
torch.manual_seed(0)

# Create the models
TwoLayerNet_model = TwoLayerNet(input_size=3*32*32, hidden_size=256, num_classes=100)
ConvNet_model = ConvNet(num_classes=100)

transform = create_transforms()
optimizer_twolayermodel = create_optimizer(TwoLayerNet_model)
optimizer_convnetmodel = create_optimizer(ConvNet_model)
criterion = torch.nn.CrossEntropyLoss()

train_dataset = CIFAR100_loader('./data', train=True, transform=transform, download=True)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

Next, train the TwoLayerNet model on the CIFAR-100 dataset using the training data loader.

In [ ]:
# YOUR CODE HERE

train(TwoLayerNet_model, train_dataloader, criterion, optimizer_twolayermodel)

Finally, train the ConvNet model on the CIFAR-100 dataset using the training data loader.

In [ ]:
# YOUR CODE HERE

train(ConvNet_model, train_dataloader, criterion, optimizer_convnetmodel)

<a id="section-7"></a>
### **Section 7: Setting Up the Hyperparameters (14 points)**

In this section, you will experiment with both the `ConvNet` and `TwoLayerNet` models by setting up and tuning the hyperparameters to achieve the highest possible accuracy. You have the flexibility to modify the training process, including the `train` function, `DataLoader`, `transform` functions, and optimizer as needed.

1. Adjust the hyperparameters, including learning rate, batch size, number of epochs, optimizer, weight decay, and transform function to improve the performance of both networks. Modify the training procedure and architecture as necessary. You can also add components like Batch Normalization layers.
2. Add two more layers to both `TwoLayerNet` and `ConvNet`. You can decide the size and placement of these layers. Evaluate if these changes result in higher performance and explain your findings.
3. Show the final results and describe the modifications made to enhance performance. Discuss the impact of hyperparameter tuning on both `TwoLayerNet` and `ConvNet`.
4. Compare the two networks in terms of architecture, performance, and learning rates. Provide a detailed explanation of the differences observed.

**Note:** Do not use external pre-trained networks and limit additional convolutional layers to a maximum of three beyond the original architecture.

In [ ]:
# YOUR CODE HERE
import optuna

# Modified TwoLayerNet architecture
class Modified_TwoLayerNet(nn.Module):

    def __init__(self, input_size, hidden_size, num_classes, dropout=0.5):
        '''
        Initializes the two-layer neural network model.

        Args:
            input_size (int): The size of the input features.
            hidden_size (int): The size of the hidden layer.
            num_classes (int): The number of classes in the dataset.
        '''

        # YOUR CODE HERE
        super(Modified_TwoLayerNet, self).__init__()

        # The modification is to add two additional linear layers and dropout/batchnorm layers

        self.flat = nn.Flatten()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.batchnorm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout1d(p=dropout)

        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.batchnorm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout1d(p=dropout)

        self.layer3 = nn.Linear(hidden_size, hidden_size)
        self.batchnorm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout1d(p=dropout)

        self.layer4 = nn.Linear(hidden_size, num_classes)

    def forward(self, x: torch.Tensor):
        '''
        Defines the forward pass of the neural network.

        Args:
            x (torch.Tensor): The input tensor.

        Returns:
            torch.Tensor: The output tensor.
        '''

        # YOUR CODE HERE

        # Apply the two layers with relu inbetween
        x = self.flat(x)

        x = self.layer1(x)
        x = self.batchnorm1(x)
        x = nn.functional.relu(x)
        x = self.dropout1(x)

        x = self.layer2(x)
        x = self.batchnorm2(x)
        x = nn.functional.relu(x)
        x = self.dropout2(x)

        x = self.layer3(x)
        x = self.batchnorm3(x)
        x = nn.functional.relu(x)
        x = self.dropout3(x)

        x = self.layer4(x)

        return x

# Modified ConvNet architecture
class Modified_ConvNet(nn.Module):

    def __init__(self, num_classes, dropout=0.5):
        '''	
        Initializes the convolutional neural network model.

        Args:
            num_classes (int): The number of classes in the dataset.
        '''

        # YOUR CODE HERE
        super(Modified_ConvNet, self).__init__()

        self.layer_C1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(5, 5))
        self.batchnorm1 = nn.BatchNorm2d(6)
        self.dropout1 = nn.Dropout2d(dropout)

        self.layer_S2 = nn.AvgPool2d(kernel_size=(2, 2))

        self.layer_C3 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5))
        self.batchnorm3 = nn.BatchNorm2d(16)
        self.dropout3 = nn.Dropout2d(dropout)

        self.layer_S4 = nn.AvgPool2d(kernel_size=(2, 2))

        self.layer_C5 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5, 5))
        self.batchnorm5 = nn.BatchNorm2d(120)
        self.dropout5 = nn.Dropout2d(dropout)

        self.flat = nn.Flatten()
        self.layer_F6 = nn.Linear(120, 84)
        self.layer_F7 = nn.Linear(84, 84)
        self.layer_F8 = nn.Linear(84, 84)

        self.gaussian_connection = nn.Linear(84, num_classes)

    def forward(self, x):
        '''
        Defines the forward pass of the neural network.

        Args:
            x (torch.Tensor): The input tensor.

        Returns:
            torch.Tensor: The output tensor.
        '''

        # YOUR CODE HERE

        # Apply all the layers and the activation functions ("sigmoid squashing function" in the paper)
        x = self.layer_C1(x)
        x = self.batchnorm1(x)
        x = nn.functional.tanh(x) * 1.7159
        x = self.dropout1(x)

        x = self.layer_S2(x)

        x = self.layer_C3(x)
        x = self.batchnorm3(x)
        x = nn.functional.tanh(x) * 1.7159
        x = self.dropout3(x)

        x = self.layer_S4(x)

        x = self.layer_C5(x)
        x = self.batchnorm5(x)
        x = nn.functional.tanh(x) * 1.7159
        x = self.dropout5(x)

        x = self.flat(x)

        x = self.layer_F6(x)
        x = nn.functional.tanh(x) * 1.7159

        x = self.layer_F7(x)
        x = nn.functional.tanh(x) * 1.7159

        x = self.layer_F8(x)
        x = nn.functional.tanh(x) * 1.7159
        
        x = self.gaussian_connection(x)

        return x

# Modified training loop. Removes the printing of the loss and accuracy
def Modified_train(net, train_loader, criterion, optimizer, epochs=100):
    '''
    Trains the neural network model.

    Args:
        net (torch.nn.Module): The neural network model.
        train_loader (torch.utils.data.DataLoader): The data loader for the training dataset.
        criterion (torch.nn.Module): The loss function.
        optimizer (torch.optim.Optimizer): The optimizer for the model.
        epochs (int): The number of epochs to train the model.

    Returns:
        tuple(list, list): The losses and accuracies for the epochs.
    '''

    # YOUR CODE HERE

    # Determine the device to run the model on
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Set the model to train mode
    net.to(device)
    net.train()

    # Store the average losses of each epoch
    losses = []
    accuracies = []

    # Train the network
    for epoch in range(epochs):

        # Store the total loss and correct predictions for the epoch to track training progress
        total_loss = 0
        correct, total = 0, 0

        for inputs, labels in train_loader:

            # Move data to correct device
            inputs, labels = inputs.to(device), labels.to(device)

            # Determine logits
            outputs = net(inputs)
            
            # Calculate the loss
            loss = criterion(outputs, labels)

            # Back propegate
            loss.backward()

            # Update weigths
            optimizer.step()

            # Remove the gradients for the next iteration
            net.zero_grad()
            optimizer.zero_grad()

            # Update the total loss and the predictions
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Store the average loss and the accuracy of the epoch
        avg_loss = total_loss / len(train_loader)
        accuracy = 100 * correct / total
        losses.append(avg_loss)
        accuracies.append(accuracy)

    return losses, accuracies

proposal_transforms = {'easy_transform': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))  
    ]),
    'crop_transform':
    transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))  
    ])}

# Define an objective function for HPO on TwoLayerNet
def objective_TwoLayerNet(trial):

    # Set seed, device and criterion
    torch.manual_seed(0)
    criterion = torch.nn.CrossEntropyLoss()

    # Get a value for all the hyper parameters
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_categorical("decay", [0.0, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2])
    transform = trial.suggest_categorical("transform", proposal_transforms)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512, 1024])
    dropout = trial.suggest_float("dropout", 0.0, 1.0)
    hidden_size = trial.suggest_categorical("hidden_size", [32, 64, 128, 256])

    # Create the model and optimizer
    model = Modified_TwoLayerNet(input_size=3*32*32, hidden_size=hidden_size, num_classes=100, dropout=dropout)
    optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Create the validation dataset
    train_dataset = CIFAR100_loader('./data', train=True, transform=proposal_transforms[transform], download=True)
    test_dataset = CIFAR100_loader('./data', train=True, transform=create_transforms(), download=True)
    validate_indices = list(range(len(train_dataset)-5000, len(train_dataset)))

    # Extract the validation set
    train_dataset = torch.utils.data.Subset(train_dataset, validate_indices)
    test_dataset = torch.utils.data.Subset(test_dataset, validate_indices)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, shuffle=False)

    train_loss, train_acc = Modified_train(model, train_dataloader, criterion, optim, epochs=25)

    accuracy = validate(model, test_dataloader)

    # Return the accuracy we wish to optimize
    return accuracy

# Define an objective function for HPO on ConvNet
def objective_ConvNet(trial):
    
    # Set seed, device and criterion
    torch.manual_seed(0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    criterion = torch.nn.CrossEntropyLoss()

    # Get a value for all the hyper parameters
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_categorical("decay", [0.0, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2])
    transform = trial.suggest_categorical("transform", proposal_transforms)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512, 1024])
    dropout = trial.suggest_float("dropout", 0.0, 1.0)

    # Create the model and optimizer
    model = Modified_ConvNet(num_classes=100, dropout=dropout)
    model.to(device)
    optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Create the validation dataset
    train_dataset = CIFAR100_loader('./data', train=True, transform=proposal_transforms[transform], download=True)
    test_dataset = CIFAR100_loader('./data', train=True, transform=create_transforms(), download=True)
    validate_indices = list(range(len(train_dataset)-5000, len(train_dataset)))

    # Extract the validation set
    train_dataset = torch.utils.data.Subset(train_dataset, validate_indices)
    test_dataset = torch.utils.data.Subset(test_dataset, validate_indices)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, shuffle=False)

    train_loss, train_acc = Modified_train(model, train_dataloader, criterion, optim, epochs=25)

    accuracy = validate(model, test_dataloader)

    # Return the accuracy we wish to optimize
    return accuracy


# Perform the study on TwoLayerNet and save the results
study_TwoLayerNet = optuna.create_study(study_name='HPO TwoLayerNet', direction="maximize")
study_TwoLayerNet.optimize(objective_TwoLayerNet, n_trials=150)
study_TwoLayerNet.trials_dataframe().to_csv('studies/TwoLayerNet.csv')
with open('studies/TwoLayerNet_BestParams.txt', 'w') as f:
    f.write(study_TwoLayerNet.best_trial.params.__str__())

# Perform the study on ConvNet and save the results
study_ConvNet = optuna.create_study(study_name='HPO ConvNet', direction="maximize")
study_ConvNet.optimize(objective_ConvNet, n_trials=150)
study_ConvNet.trials_dataframe().to_csv('studies/ConvNet.csv')
with open('studies/ConvNet_BestParams.txt', 'w') as f:
    f.write(study_ConvNet.best_trial.params.__str__())



Test the performance of TwoLayerNet after hyperparameter tuning and compare it with the ConvNet model. Provide a detailed explanation of the results.

In [ ]:
# YOUR CODE HERE


Test the performance of ConvNet after hyperparameter tuning and compare it with the TwoLayerNet model. Provide a detailed explanation of the results.

In [ ]:
# YOUR CODE HERE

<a id="section-8"></a>
### **Section 8: Visualizing the STL-10 Dataset and Preparing the Data Loader (3 points)**

In this section, you will work with a subset of the [STL-10](https://cs.stanford.edu/~acoates/stl10/) dataset, containing higher resolution images and different object classes than CIFAR-100. Before fine-tuning your ConvNet on this dataset, first complete the `visualise_stl10` function to display sample images from the following 5 classes:

1. **Bird**
2. **Deer**
3. **Dog**
4. **Horse**
5. **Monkey**

In [ ]:
def visualise_stl10(class_mapping):
    '''
    Visualizes 5 images from each specified class in the STL-10 dataset.

    Args:
        class_mapping (dict): A dictionary mapping class indices to class names.
    '''

    # YOUR CODE HERE

In [ ]:
# Define the class mapping for bird, deer, dog, horse, and monkey
class_mapping = {1: 'bird', 4: 'deer', 5: 'dog', 6: 'horse', 7: 'monkey'}

# Visualize STL-10 classes
visualise_stl10(class_mapping)

After visualizing the data, implement the `STL10_loader` class to create a custom data loader that initializes the dataset, extracts the target classes, and applies the necessary image transformations. Once these tasks are completed, you will move on to fine-tuning the ConvNet on this dataset in the next section.

In [ ]:
class STL10_loader(Dataset):
    def __init__(self, root, train=True, transform=None):
        '''
        Initializes the STL10 dataset.

        Args:
            root (str): Root directory of the dataset.
            train (bool): If True, use the training set, otherwise use the test set.
            transform (callable, optional): A function/transform to apply to the images.
        '''

        # YOUR CODE HERE
        self.dataset = torchvision.datasets.STL10()
        
    def __len__(self):
        '''
        Returns the number of samples in the dataset.
        '''

        # YOUR CODE HERE

    def __getitem__(self, idx):
        '''
        Retrieves a sample from the dataset at the specified index.

        Args:
            idx (int): The index of the sample to retrieve.

        Returns:
            tuple: A tuple containing the transformed image and its target label.
        '''

        # YOUR CODE HERE

<a id="section-9"></a>
### **Section 9: Fine-tuning ConvNet on STL-10 (14 points)**

In this section, you will load the pre-trained parameters of the ConvNet (trained on CIFAR-100) and modify the output layer to adapt it to the new dataset containing 5 classes. You can either first load the pre-trained parameters and then modify the output layer, or change the output layer before loading the matched pre-trained parameters. Once modified, you will train the model and document the settings of hyperparameters, accuracy, and learning curve. Additionally, visualize both the training loss and accuracy to assess the learning process. To gain a deeper understanding of the feature learning process, consider using techniques like [**t-sne**](https://lvdmaaten.github.io/tsne/) for feature space visualization.

In [ ]:
# YOUR CODE HERE

<a id="section-10"></a>
### **Section 10: Bonus Challenge (optional)**

Try to achieve the highest possible accuracy on the test dataset (5 classes from STL-10) by adjusting hyperparameters, modifying architectures, or applying techniques like data augmentation. The top-performing teams will earn bonus points that can significantly boost their final lab grade, even allowing it to exceed 10 (up to 11):

- **1st place:** +1.0 to the final grade of the final lab
- **2nd place:** +0.8 to the final grade of the final lab
- **3rd place:** +0.6 to the final grade of the final lab
- **4th place:** +0.4 to the final grade of the final lab
- **5th place:** +0.2 to the final grade of the final lab

**Hint:** You may use techniques like data augmentation, freezing early layers, modifying architecture, or optimizing hyperparameters. Only data from CIFAR-100 and STL-10 can be used, and you cannot add more than 3 additional convolutional layers.

In [ ]:
# YOUR CODE HERE

<a id="section-x"></a>
### **Section X: Individual Contribution Report *(Mandatory)***

Because we want each student to contribute fairly to the submitted work, we ask you to fill out the textcells below. Write down your contribution to each of the assignment components in percentages. Naturally, percentages for one particular component should add up to 100% (e.g. 30% - 30% - 40%). No further explanation has to be given.

| Name | Contribution on Research | Contribution on Programming | Contribution on Writing |
| -------- | ------- | ------- | ------- |
|  | - % | - % | - % |
|  | - % | - % | - % |
|  | - % | - % | - % |
|  | - % | - % | - % |

### - End of Notebook -

In [ ]:
%pip list

In [ ]:
%pip install torchinfo